In [1]:

import pandas as pd
import numpy as np
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import os
os.environ['R_HOME'] = '/root/miniconda3/envs/R/lib/R'

import rpy2.robjects as objects
from rpy2.robjects.packages import importr
base = importr('base')
r_pROC = importr("pROC")
base._libPaths()[0]

def df_stats(df):
    from tabulate import tabulate
    print("\n***** Shape: ", df.shape," *****\n")
    
    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()
    
    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(list_stat_val, columns=['Name', 'Null', 'Unique', 'Dtypes'])
    print(tabulate(df_stat_val, headers='keys', tablefmt='psql'))
    return df.head()

/tmp/ipykernel_273258/3190050256.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
df_lqts = pd.read_csv('data/ecg_lqts_rafik.csv')

/tmp/ipykernel_273258/4203077839.py:1: DtypeWarning: Columns (18,34,109,110,177,178,180,188,346,352,364,410,432) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lqts = pd.read_csv('data/ecg_lqts_rafik.csv')


In [3]:
## Get filename from xml_path
df_lqts['xml_filename'] = df_lqts['xml_path'].apply(lambda x: x.split('/')[-1])
display(df_lqts.describe())

,Unnamed: 0,RestingECG_PatientDemographics_PatientID,RestingECG_PatientDemographics_PatientAge,RestingECG_TestDemographics_Site,RestingECG_TestDemographics_Location,RestingECG_TestDemographics_CartNumber,RestingECG_TestDemographics_AnalysisSoftwareVersion,RestingECG_TestDemographics_AcquisitionTechID,RestingECG_TestDemographics_EditorID,RestingECG_TestDemographics_HISStatus,RestingECG_RestingECGMeasurements_VentricularRate,RestingECG_RestingECGMeasurements_AtrialRate,RestingECG_RestingECGMeasurements_PRInterval,RestingECG_RestingECGMeasurements_QRSDuration,RestingECG_RestingECGMeasurements_QTInterval,RestingECG_RestingECGMeasurements_QTCorrected,RestingECG_RestingECGMeasurements_PAxis,RestingECG_RestingECGMeasurements_RAxis,RestingECG_RestingECGMeasurements_TAxis,RestingECG_RestingECGMeasurements_QRSCount,RestingECG_RestingECGMeasurements_QOnset,RestingECG_RestingECGMeasurements_QOffset,RestingECG_RestingECGMeasurements_POnset,RestingECG_RestingECGMeasurements_POffset,RestingECG_RestingECGMeasurements_TOffset,RestingECG_RestingECGMeasurements_ECGSampleBase,RestingECG_RestingECGMeasurements_ECGSampleExponent,RestingECG_RestingECGMeasurements_QTcFrederica,RestingECG_OriginalRestingECGMeasurements_VentricularRate,RestingECG_OriginalRestingECGMeasurements_AtrialRate,RestingECG_OriginalRestingECGMeasurements_PRInterval,RestingECG_OriginalRestingECGMeasurements_QRSDuration,RestingECG_OriginalRestingECGMeasurements_QTInterval,RestingECG_OriginalRestingECGMeasurements_QTCorrected,RestingECG_OriginalRestingECGMeasurements_PAxis,RestingECG_OriginalRestingECGMeasurements_RAxis,RestingECG_OriginalRestingECGMeasurements_TAxis,RestingECG_OriginalRestingECGMeasurements_QRSCount,RestingECG_OriginalRestingECGMeasurements_QOnset,RestingECG_OriginalRestingECGMeasurements_QOffset,RestingECG_OriginalRestingECGMeasurements_POnset,RestingECG_OriginalRestingECGMeasurements_POffset,RestingECG_OriginalRestingECGMeasurements_TOffset,RestingECG_OriginalRestingECGMeasurements_ECGSampleBase,RestingECG_OriginalRestingECGMeasurements_ECGSampleExponent,RestingECG_OriginalRestingECGMeasurements_QTcFrederica,RestingECG_QRSTimesTypes_QRS_0_Number,RestingECG_QRSTimesTypes_QRS_0_Type,RestingECG_QRSTimesTypes_QRS_0_Time,RestingECG_QRSTimesTypes_QRS_1_Number,RestingECG_QRSTimesTypes_QRS_1_Type,RestingECG_QRSTimesTypes_QRS_1_Time,RestingECG_QRSTimesTypes_QRS_2_Number,RestingECG_QRSTimesTypes_QRS_2_Type,RestingECG_QRSTimesTypes_QRS_2_Time,RestingECG_QRSTimesTypes_QRS_3_Number,RestingECG_QRSTimesTypes_QRS_3_Type,RestingECG_QRSTimesTypes_QRS_3_Time,RestingECG_QRSTimesTypes_QRS_4_Number,RestingECG_QRSTimesTypes_QRS_4_Type,RestingECG_QRSTimesTypes_QRS_4_Time,RestingECG_QRSTimesTypes_QRS_5_Number,RestingECG_QRSTimesTypes_QRS_5_Type,RestingECG_QRSTimesTypes_QRS_5_Time,RestingECG_QRSTimesTypes_QRS_6_Number,RestingECG_QRSTimesTypes_QRS_6_Type,RestingECG_QRSTimesTypes_QRS_6_Time,RestingECG_QRSTimesTypes_QRS_7_Number,RestingECG_QRSTimesTypes_QRS_7_Type,RestingECG_QRSTimesTypes_QRS_7_Time,RestingECG_QRSTimesTypes_QRS_8_Number,RestingECG_QRSTimesTypes_QRS_8_Type,RestingECG_QRSTimesTypes_QRS_8_Time,RestingECG_QRSTimesTypes_QRS_9_Number,RestingECG_QRSTimesTypes_QRS_9_Type,RestingECG_QRSTimesTypes_QRS_9_Time,RestingECG_QRSTimesTypes_QRS_10_Number,RestingECG_QRSTimesTypes_QRS_10_Type,RestingECG_QRSTimesTypes_QRS_10_Time,RestingECG_QRSTimesTypes_QRS_11_Number,RestingECG_QRSTimesTypes_QRS_11_Type,RestingECG_QRSTimesTypes_QRS_11_Time,RestingECG_QRSTimesTypes_QRS_12_Number,RestingECG_QRSTimesTypes_QRS_12_Type,RestingECG_QRSTimesTypes_QRS_12_Time,RestingECG_QRSTimesTypes_QRS_13_Number,RestingECG_QRSTimesTypes_QRS_13_Type,RestingECG_QRSTimesTypes_QRS_13_Time,RestingECG_QRSTimesTypes_GlobalRR,RestingECG_QRSTimesTypes_QTRGGR,RestingECG_QRSTimesTypes_QRS_14_Number,RestingECG_QRSTimesTypes_QRS_14_Type,RestingECG_QRSTimesTypes_QRS_14_Time,RestingECG_TestDemographics_OverreaderID,RestingECG_Order_RequisitionNumber,RestingECG_Order_AdmissionType,RestingECG_RestingECGMeasurements_SystolicBP,Restin

In [21]:

## Copy df_lqts['xml_path'] to 'data/lqts_ecg/' folder

import os
import shutil

def copy_files(df, src, dst):
# Make dst directory if it doesn't exist

    if not os.path.exists(dst):
        os.makedirs(dst)

    for index, row in df.iterrows():
        file_name = row['xml_filename']
        dst_file = os.path.join(dst, file_name)
        shutil.copyfile(row['xml_path'], dst_file)

copy_files(df_lqts, 'data/lqts_ecg/', 'data/lqts_ecg/')

KeyboardInterrupt: 

In [ ]:
import lqtnet
#Run script "python lqtnet/extract_ecg_xml.py --source_dir data/lqts_ecg --dest_dir data/ECG_CSV" in the notebook
!python extract_ecg_xml.py --source_dir data/lqts_ecg --dest_dir data/ECG_CSV


Saved: data/ECG_CSV/MUSE_20221109_143926_79000.csv
Saved: data/ECG_CSV/MUSE_20221208_193827_67000.csv
Saved: data/ECG_CSV/MUSE_20221111_132755_11000.csv
Saved: data/ECG_CSV/MUSE_20221109_004315_62000.csv
Saved: data/ECG_CSV/MUSE_20221112_185535_15000.csv
Saved: data/ECG_CSV/MUSE_20221201_013244_30000.csv
Saved: data/ECG_CSV/MUSE_20220912_202203_09000.csv
Saved: data/ECG_CSV/MUSE_20221207_194821_13000.csv
Saved: data/ECG_CSV/MUSE_20221129_185132_74000.csv
Saved: data/ECG_CSV/MUSE_20221210_215823_77000.csv
Saved: data/ECG_CSV/MUSE_20221110_194724_77000.csv
Saved: data/ECG_CSV/MUSE_20221204_145636_48000.csv
Saved: data/ECG_CSV/MUSE_20221111_073734_64000.csv
Saved: data/ECG_CSV/MUSE_20221202_180442_54000.csv
Saved: data/ECG_CSV/MUSE_20221201_144522_12000.csv
Saved: data/ECG_CSV/MUSE_20221130_134917_49000.csv
Saved: data/ECG_CSV/MUSE_20221115_172744_49000.csv
Saved: data/ECG_CSV/MUSE_20220831_125702_86000.csv
Saved: data/ECG_CSV/MUSE_20221211_232013_57000.csv
Saved: data/ECG_CSV/MUSE_202212

In [ ]:
## Get filename from xml_path
df_lqts['xml_filename'] = df_lqts['xml_path'].apply(lambda x: x.split('/')[-1])

df_lqts = df_lqts.rename(columns={
    'xml_path': 'xml_path',
    'RestingECG_PatientDemographics_PatientID': 'patient_id',
    'old_id_site_column': 'id_site',
    'RestingECG_TestDemographics_RoomID': 'id_site',
    'RestingECG_PatientDemographics_DateofBirth': 'dob',
    'RestingECG_PatientDemographics_PatientAge': 'age',
    'RestingECG_PatientDemographics_Gender': 'sex',
    'RestingECG_Order_OrderPlacementDate': 'date',
    'RestingECG_Order_OrderPlacementTime': 'hr',
    'RestingECG_RestingECGMeasurements_QTCorrected': 'qt_manual',
    'RestingECG_OriginalRestingECGMeasurements_QTCorrected': 'qt_manual_confirmed'
})

df_lqts['set'] = 'External validation'
df_lqts['ethnicity'] = 'Unknown'
df_lqts['qc'] = 'False'
## Create df_lqts['file'] by adding the df_lqts['xml_filename'] to 'data/ECG_CSV' but change filepath from xml to .CSV
df_lqts['file'] = 'data/ECG_CSV/' + df_lqts['xml_filename'].apply(lambda x: x.replace('.xml', '.csv'))


In [ ]:
## Check if df_lqts['file'] exists
df_lqts['file_exists'] = df_lqts['file'].apply(lambda x: os.path.exists(x))
display(df_lqts['file_exists'].value_counts())

True    2050
Name: file_exists, dtype: int64

In [5]:
df_lqts_rafik = pd.read_csv('../DeepECG/data/LQTS/DeepECG-LQT-20230210.csv', delim_whitespace=True)
## Check if df_LQTS['Dossier'] is in df_ecg['RestingECG_PatientDemographics_PatientID']
df_lqts_rafik['Dossier'] = df_lqts_rafik['Dossier'].astype(int)
## Convert Genotype : Negative = Control ; LQT1 = Type 1 or LQT2 = Type 2
df_lqts_rafik['Genotype'] = df_lqts_rafik['Genotype'].apply(lambda x: 'Control' if x == 'Negative' else x)
df_lqts_rafik['Genotype'] = df_lqts_rafik['Genotype'].apply(lambda x: 'Type 1' if x == 'LQT1' else x)
df_lqts_rafik['Genotype'] = df_lqts_rafik['Genotype'].apply(lambda x: 'Type 2' if x == 'LQT2' else x)
#REname column Genotype to lqts_type
df_lqts_rafik = df_lqts_rafik.rename(columns={'Genotype': 'lqts_type'})
display(df_lqts_rafik.lqts_type.value_counts())

Control    153
Type 1      28
Type 2      18
Name: lqts_type, dtype: int64

In [ ]:
#df_lqts_merged = pd.merge(df_lqts, df_lqts_rafik, how='left', left_on='patient_id', right_on='Dossier')
#df_lqts_merged.to_csv('data/df_lqts_merged.csv')

In [29]:
df_lqts_merged = pd.read_csv('data/df_lqts_merged.csv')
## df_lqts_merged replace 'data' with '/media/data1/ravram/DeepECG/'
df_lqts_merged['npy_path'] = df_lqts_merged['npy_path'].apply(lambda x: x.replace('data', '/media/data1/ravram/DeepECG/'))
## Check if df_lqts_merged['npy_path'] exists
df_lqts_merged['npy_path_exists'] = df_lqts_merged['npy_path'].apply(lambda x: os.path.exists(x))
display(df_lqts_merged['npy_path_exists'].value_counts())
## Load df_lqts_merged npy_path for each row and add the shape of the array to df_lqts_merged['npy_shape']

df_lqts_merged['npy_shape'] = df_lqts_merged['npy_path'].apply(lambda x: np.load(x).shape)
#df_lqts_merged['npy_shape'] = df_lqts_merged['npy_shape'].apply(lambda x: x[0])


/tmp/ipykernel_70253/3462177575.py:1: DtypeWarning: Columns (19,35,110,111,178,179,181,189,347,353,365,411,433) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lqts_merged = pd.read_csv('data/df_lqts_merged.csv')


True    2050
Name: npy_path_exists, dtype: int64

### Extract files

In [1]:
import lqtnet.sierraecg
import lqtnet.import_ecgs
import lqtnet.train

2023-03-15 21:05:33.535209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
import sys
import os

import pandas as pd
import numpy as np
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



dir2 = os.path.abspath("/volume/lqts/lqtnet")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)


import lqtnet
import lqtnet.import_ecgs

# directory containing normalized CSV files
ECG_SOURCE_DIR = ''
MODEL_PATH = '/volume/lqts/models/2023.02.26-1/'

metadata = pd.read_csv('data/df_lqts_merged.csv')
ext_df = metadata.query('set == "External validation"')

x_ext, erronous_files = lqtnet.import_ecgs.df_import_csv_to_numpy(ext_df, from_dir=ECG_SOURCE_DIR)
y_ext = lqtnet.import_ecgs.df_to_np_labels(ext_df)
print("Loading model from {}".format(MODEL_PATH))
model = lqtnet.train._load_model(MODEL_PATH)
print("Making predictions")
# make predictions - save this output for further analysis
y_extval_pred = model.predict(x_ext)

/tmp/ipykernel_125226/522757672.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/tmp/ipykernel_125226/522757672.py:29: DtypeWarning: Columns (19,35,110,111,178,179,181,189,347,353,365,411,433) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('data/df_lqts_merged.csv')


Loading model from /volume/lqts/models/2023.02.26-1/
Making predictions
64/64 [==============================] - 1s 15ms/step


In [6]:
# if metadata['file'] is in erronous_files then remove it from metadata
display(len(metadata))

display(len(metadata))

2050

2037

In [29]:
#metadata['y_true_bin'] = label_binarize(metadata['lqts_type'], classes=np.arange(n_classes))


ValueError: Expected a 1D array, got an array with shape (2037, 3)

In [20]:
## Pick max value from y_extval_pred and convert it to integer 0, 1 or 2 depending to column
#y_extval_pred = np.argmax(y_extval_pred, axis=1)

# Create column in metadata called y_pred and populate it with y_extval_pred
#metadata['y_pred'] = y_extval_pred


In [40]:
display(metadata.lqts_type.value_counts())

Control    1522
Type 1      313
Type 2      202
Name: lqts_type, dtype: int64

In [42]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# Remove erronous files from metadata
metadata = metadata[~metadata['file'].isin(erronous_files)]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 3
lw = 2


# Binarize the true labels
#n_classes = len(np.unique(metadata['lqts_type']))
#metadata['y_true_bin'] = label_binarize(metadata['lqts_type'], classes=np.arange(n_classes))

# Compute multi-class AUC using One-vs-One (ovo) strategy
auc_ovo = roc_auc_score(metadata['lqts_type'], y_extval_pred, multi_class='ovo')

# Compute multi-class AUC using One-vs-Rest (ovr) strategy
auc_ovr = roc_auc_score(metadata['lqts_type'], y_extval_pred, multi_class='ovr')

In [56]:
## Groupby metadata by patient_id and keep lqts_type and mean of argmax of "y_pred_1", "y_pred_2", "y_pred_3"
metadata_grouped = metadata.groupby('patient_id').agg({'lqts_type': 'first', 'y_pred_0': 'mean', 'y_pred_1': 'mean', 'y_pred_2': 'mean'})
metadata_grouped['y_pred'] = np.argmax(metadata_grouped[['y_pred_0','y_pred_1', 'y_pred_2']].values, axis=1)


# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 3
lw = 2


# Binarize the true labels
#n_classes = len(np.unique(metadata['lqts_type']))
#metadata['y_true_bin'] = label_binarize(metadata['lqts_type'], classes=np.arange(n_classes))

# Compute multi-class AUC using One-vs-One (ovo) strategy
auc_ovo = roc_auc_score(metadata_grouped['lqts_type'], metadata_grouped[['y_pred_0','y_pred_1','y_pred_2']], multi_class='ovo')
print(auc_ovo)
# Compute multi-class AUC using One-vs-Rest (ovr) strategy
auc_ovr = roc_auc_score(metadata_grouped['lqts_type'], metadata_grouped[['y_pred_0','y_pred_1','y_pred_2']], multi_class='ovr')
print(auc_ovr)

0.7872262087948362
0.7862496342914714


In [37]:
metadata['y_pred_0'], metadata['y_pred_1'], metadata['y_pred_2'] = y_extval_pred[:,0], y_extval_pred[:,1], y_extval_pred[:,2]
metadata['y_pred'] = np.argmax(y_extval_pred, axis=1)


In [46]:
## Remove RestingECG_PatientDemographics_PatientFirstName, RestingECG_PatientDemographics_PatientLastName, dob froom metadata
metadata = metadata.drop(['RestingECG_PatientDemographics_PatientFirstName', 'RestingECG_PatientDemographics_PatientLastName', 'dob'], axis=1)

KeyError: "['RestingECG_PatientDemographics_PatientFirstName', 'RestingECG_PatientDemographics_PatientLastName', 'dob'] not found in axis"

In [47]:
metadata.to_csv('data/LQTS_predictions.csv')

displa